In [ ]:
%%bash
# download Pfam HMM-profiles
wget https://ftp.ebi.ac.uk/pub/databases/Pfam/current_release/Pfam-A.hmm.gz
gunzip Pfam-A.hmm.gz

# install HMMER3
apt install hmmer

In [ ]:
!pip install pyhmmer biopython

import re
import pandas as pd

from pyhmmer.easel import SequenceFile
from pyhmmer.hmmer import HMMFile, hmmscan

from Bio import SeqIO

In [ ]:
data = pd.read_csv('For project - selected.csv')
data.head()

,HGNC approved symbol,Product,Function,Protein complex,Target entity,HGNC approved name,UniProt ID (human),Pfam domains,UniProt ID (mouse),Modification
0,A1CF,U,RNA modification,APOB_mRNA_editosome,"mRNA, mC",APOBEC1 complementation factor,A1CF_HUMAN,"DND1_DSRM PF14709 445-523, RRM_1 PF00076 58-12...",A1CF_MOUSE,RNA deamination
1,ACINU,#,RNA modification,#,mRNA,Apoptotic chromatin condensation inducer in th...,ACINU_HUMAN,PF16294;PF02037,ACINU_MOUSE,Alternative splicing
2,ACTB,#,Chromatin remodeling cofactor,"BAF, nBAF, npBAF, PBAF, SWI/SNF-like EPAFB, bB...",#,"actin, beta",ACTB_HUMAN,Actin PF00022 2-375,ACTB_MOUSE,#
3,ACTL6A,#,Chromatin remodeling cofactor,"BAF, npBAF, PBAF, SWI/SNF_Brg1(I), SWI/SNF_Brg...",#,actin-like 6A,ACL6A_HUMAN,Actin PF00022 8-429,ACL6A_MOUSE,#
4,ACTL6B,#,Chromatin remodeling cofactor,"BAF, nBAF, PBAF, SWI/SNF_Brg1(I), SWI/SNF_Brg1...",#,actin-like 6B,ACL6B_HUMAN,Actin PF00022 8-426,ACL6B_MOUSE,#


In [ ]:
# Get unique Pfam IDs from "For project - selected.csv"
query = set(re.findall(r'PF\d{5}', ' '.join(data['Pfam domains'])))
len(query)

514

In [ ]:
available_ids = []

# Convert Pfam IDs (PFXXXXX.X -> PFXXXXX)
with open('Pfam-A.hmm') as file1, open('Pfam-A[converted].hmm', 'w') as file2:
    for line in map(lambda s: s.removesuffix('\n'), file1):
        if line.startswith('ACC'):
            _, pfam_id = line.split()
            pfam_id = pfam_id.split('.')[0]
            available_ids.append(pfam_id)

            print(f'ACC   {pfam_id}', file=file2)
        else:
            print(line, file=file2)

In [ ]:
# Filter query and save to file
query = [pfam_id for pfam_id in query if pfam_id in available_ids]

with open('query.txt', 'w') as file:
    print('\n'.join(query), file=file)

len(query)

510

In [ ]:
%%bash
# Fetch subset
hmmfetch --index Pfam-A[converted].hmm
hmmfetch -f Pfam-A[converted].hmm query.txt > Pfam-subset.hmm

Working...    done.
Indexed 21979 HMMs (21979 names and 21979 accessions).
SSI index written to file Pfam-A[converted].hmm.ssi


In [ ]:
!gunzip UP000008153_5671.fasta.gz

In [ ]:
seqfile = 'UP000008153_5671.fasta'
sequences = SeqIO.index(seqfile, 'fasta')

# Run hmmscan
with (
    SequenceFile(seqfile, digital=True) as queries,
    HMMFile('Pfam-subset.hmm') as profiles,
    open('output.tsv', mode='w') as tsv_outfile,
    open('output.faa', mode='w') as faa_outfile
):
    for hits in hmmscan(queries, profiles):
        if hits:
            seq_id = hits.query_name.decode()

            print(seq_id, *[hit.accession.decode() for hit in hits], sep='\t', file=tsv_outfile)
            SeqIO.write(sequences[seq_id], faa_outfile, 'fasta')

In [1]:
!pip install pandas gffutils


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 14.5 MB/s eta 0:00:00


In [24]:
import pandas as pd
import gffutils

def read_and_split_z_dna_file(file_path):
    data = []
    with open(file_path, 'r') as file:
        for line in file:
            # Пропускаем строку, если это заголовок или пустая строка
            line = line.strip()
            if not line or line.lower().startswith('start'):
                continue

            # Разбиваем строку на два значения
            parts = line.split()
            if len(parts) == 2:
                start, end = parts
                data.append({'start': int(start), 'end': int(end)})
            else:
                print(f"Skipping malformed line: {line}")

    return pd.DataFrame(data)

z_dna_df = read_and_split_z_dna_file('z_dna.txt')

Skipping malformed line: NC_009387.2
Skipping malformed line: NC_009388.2
Skipping malformed line: NC_009389.2
Skipping malformed line: NC_009390.2
Skipping malformed line: NC_009277.2
Skipping malformed line: NC_009391.2
Skipping malformed line: NC_009392.2
Skipping malformed line: NC_009393.2
Skipping malformed line: NC_009394.2
Skipping malformed line: NC_009395.2
Skipping malformed line: NC_009396.2
Skipping malformed line: NC_009397.2
Skipping malformed line: NC_009398.2
Skipping malformed line: NC_009399.2
Skipping malformed line: NC_009400.2
Skipping malformed line: NC_009401.2
Skipping malformed line: NC_009402.2
Skipping malformed line: NC_009403.2
Skipping malformed line: NC_009404.2
Skipping malformed line: NC_009405.2
Skipping malformed line: NC_009406.2
Skipping malformed line: NC_009407.2
Skipping malformed line: NC_009408.2
Skipping malformed line: NC_009409.2
Skipping malformed line: NC_009410.2
Skipping malformed line: NC_009411.2
Skipping malformed line: NC_009412.2
S

In [ ]:
def process_z_dna_file(z_dna_file):
    z_dna_coordinates = []

    # Читаем файл с координатами Z-DНК
    with open(z_dna_file, 'r') as f:
        for line in f:
            # Пропускаем строки, начинающиеся с "NW" или содержащие слово "start"
            if line.startswith('NW') or ' start ' in line:
                continue

            # Разделяем строку на части
            parts = line.strip().split()

            # Пропускаем строки, которые не разделились на две части (например, заголовки)
            if len(parts) != 2:
                print(f"Skipping invalid line: {line}")
                continue

            # Пытаемся преобразовать начальную и конечную координату в целые числа
            try:
                start = int(parts[0])
                end = int(parts[1])
                z_dna_coordinates.append((start, end))
            except ValueError:
                print(f"Skipping line with non-integer coordinates: {line}")

    return z_dna_coordinates

# Пример использования функции
z_dna_file = 'z_dna.txt'
z_dna_coordinates = process_z_dna_file(z_dna_file)

# Выводим координаты Z-DНК для проверки
#for start, end in z_dna_coordinates:
    #print(f"Start: {start}, End: {end}")

print(type(z_dna_coordinates))

In [61]:
import gffutils

# Указать путь к файлу GTF
gtf_file = 'genomic.gtf'

# Создать базу данных SQLite из файла GTF
db = gffutils.create_db(gtf_file, dbfn='annotation.db', force=True, merge_strategy='merge')

# Открываем созданную базу данных
db = gffutils.FeatureDB('annotation.db')

# Пример использования базы данных (запрос всех генов)



# Закрываем базу данных
#db.close()


In [64]:
gtf_file = 'genomic.gtf'
output_file = 'genome_annotation_without_header.gtf'

# Открываем исходный файл GTF и создаем новый файл без первых трех строк
with open(gtf_file, 'r') as f_in, open(output_file, 'w') as f_out:
    # Пропускаем первые три строки
    for i, line in enumerate(f_in):
        if i < 3:
            continue
        f_out.write(line)

print(f"Removed headers from {gtf_file}. Output written to {output_file}")


Removed headers from genomic.gtf. Output written to genome_annotation_without_header.gtf


In [82]:
import gffutils

# Функция для определения типа региона генома
def determine_genomic_region(feature, start, end):
    if feature.featuretype == 'gene':
        return 'gene'
    elif feature.featuretype == 'exon' and feature.start <= start and feature.end >= end:
        return 'exon'
    elif feature.featuretype == 'intron' and feature.start <= start and feature.end >= end:
        return 'intron'
    elif feature.start < start < feature.end and end > feature.end:
        return 'downstream'
    elif start < feature.start < end < feature.end:
        return 'upstream'
    elif feature.start - start <= 1000 and feature.end >= start:
        return 'promoter'
    else:
        return 'intergenic'

# Путь к файлу GFF аннотации генома
gff_file = 'genomic.gff'

# Создание базы данных из файла GFF, если она еще не создана
db = gffutils.create_db(gff_file, dbfn='annotation.db', force=True, merge_strategy='merge')

# Открываем созданную базу данных
db = gffutils.FeatureDB('annotation.db')

# Путь к файлу с координатами Z-ДНК
zdna_file = 'g4.txt'

# Открываем файл с координатами Z-ДНК
with open(zdna_file, 'r') as f_in, open('q4_with_regions.txt', 'w') as f_out:
    f_out.write("start\tend\tregion\n")  # Заголовок файла вывода

    for line in f_in:
        if line.startswith('start'):  # Пропускаем строку с заголовком
            continue
        # Пропускаем строки, начинающиеся с "NW" или содержащие слово "start"
        if line.startswith('NW') or ' start ' in line:
            continue
        if line.startswith('NC') or ' start ' in line:
            continue
        start, end = map(int, line.strip().split())

        # Ищем пересечения с Z-ДНК в базе данных аннотации генома
        for feature in db.region(start=start, end=end, completely_within=False):
            region_type = determine_genomic_region(feature, start, end)
            f_out.write(f"{start}\t{end}\t{region_type}\n")

# Закрываем базу данных
db.close()

print("Расчет завершен. Результаты записаны в файл g4_with_regions.txt")


AttributeError: 'FeatureDB' object has no attribute 'close'

In [90]:
# Путь к исходному файлу с результатами
input_file = 'q4_with_regions.txt'

# Путь к файлу, в который будем записывать уникальные строки
output_file = 'q4_unique_regions.txt'

# Множество для отслеживания уникальных строк
unique_lines = set()

# Чтение и запись уникальных строк
with open(input_file, 'r') as f_in, open(output_file, 'w') as f_out:
    header = f_in.readline()  # Читаем заголовок и записываем в новый файл
    f_out.write(header)

    for line in f_in:
        if line.strip() not in unique_lines:
            unique_lines.add(line.strip())
            f_out.write(line)

print("Уникальные строки записаны в файл Z_DNA_unique_regions.txt")


Уникальные строки записаны в файл Z_DNA_unique_regions.txt


In [91]:
# Путь к файлу с результатами
result_file = 'q4_unique_regions.txt'

# Словарь для подсчета количества Z-ДНК в каждой категории
counts = {
    'promoter': 0,
    'gene': 0,
    'exon': 0,
    'intron': 0,
    'upstream': 0,
    'downstream': 0,
    'intergenic': 0
}
total=0
# Чтение файла с результатами
with open(result_file, 'r') as f:
    next(f)  # Пропускаем заголовок
    for line in f:
        start, end, region_type = line.strip().split('\t')
        counts[region_type] += 1
        total+=1
# Вывод результатов
print(f"Количество g4-ДНК в промоторах: {counts['promoter']}")
print(f"Количество g4-ДНК в генах: {counts['gene']}")
print(f"Количество g4-ДНК в экзонах: {counts['exon']}")
print(f"Количество g4-ДНК в интронах: {counts['intron']}")
print(f"Количество g4-ДНК в upstream: {counts['upstream']}")
print(f"Количество g4-ДНК в downstream: {counts['downstream']}")
print(f"Количество g4-ДНК в межгенном пространстве: {counts['intergenic']}")
print(total)

Количество g4-ДНК в промоторах: 9510
Количество g4-ДНК в генах: 9278
Количество g4-ДНК в экзонах: 266
Количество g4-ДНК в интронах: 0
Количество g4-ДНК в upstream: 1571
Количество g4-ДНК в downstream: 1621
Количество g4-ДНК в межгенном пространстве: 0
22246


In [79]:
! pip install Bio
from Bio import SeqIO
import re
sequences = SeqIO.parse("genome.fna", "fasta")
pattern = "(?:G{3,}[ATGC]{1,7}){3,}G{3,}"
pattern_C = "(?:C{3,}[ATGC]{1,7}){3,}C{3,}"
for seq in sequences:
    PQS_plus = [[m.start(), m.end(), m.group(0)] for m in re.finditer(pattern,
str(seq.seq), re.IGNORECASE)]
    PQS_minus = [[m.start(), m.end(), m.group(0)] for m in re.finditer(pattern_C,
str(seq.seq), re.IGNORECASE)]
    with open("g4.txt", "w") as f:
      f.write("0\t1\n")
      for i in (PQS_minus + PQS_plus):
        f.write(f"{i[0]}\t{i[1]}\n")
print(len(PQS_plus) + len(PQS_minus))


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.0/281.0 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 10.1 MB/s eta 0:00:00
3


In [84]:
! head /content/q4_with_regions.txt

start	end	region
0	178	upstream
0	178	upstream
0	178	upstream
0	178	upstream
0	178	upstream
0	178	upstream
0	178	upstream
0	178	upstream
0	178	upstream
